In [28]:
import numpy as np
import pandas as pd
import glob, os


# Usage

Just select the correct pattern of the file/folders that you want to find and then write the output in a file for using with bash

### Example
find all the HorizonDump.h5 and output the file paths with a space in between. 

JoinH5 -o combined.h5 <paste_the_file_content>


# Code

In [26]:
folder_path = "/panfs/ds09/sxs/himanshu/runs/simple_BBH_for_GH_testing/one_high_spin_zero_ecc_equal_mass"

# # With ringdowns
# lev_golb=folder_path+"/Ev/Lev**"

# Without ringdowns
lev_golb=folder_path+"/Ev/Lev?_??"



checkpoints_glob=lev_golb+"/Run/Checkpoints/?**"
checkpoints_A0_glob=lev_golb+"/Run/Checkpoints/?**/Cp-VarsGr_SphereA0**.h5"

horizon_dump_glob=lev_golb+"/Run/ApparentHorizons/HorizonsDump.h5"




In [27]:

path_pattern = checkpoints_A0_glob
path_collection = []


for folder_name in glob.iglob(path_pattern, recursive=True):
    if os.path.isdir(folder_name) or os.path.isfile(folder_name):
        path_collection.append(folder_name)
        print(folder_name)

/panfs/ds09/sxs/himanshu/runs/simple_BBH_for_GH_testing/one_high_spin_zero_ecc_equal_mass/Ev/Lev1_AA/Run/Checkpoints/6806/Cp-VarsGr_SphereA0.h5
/panfs/ds09/sxs/himanshu/runs/simple_BBH_for_GH_testing/one_high_spin_zero_ecc_equal_mass/Ev/Lev1_AA/Run/Checkpoints/10698/Cp-VarsGr_SphereA0.h5
/panfs/ds09/sxs/himanshu/runs/simple_BBH_for_GH_testing/one_high_spin_zero_ecc_equal_mass/Ev/Lev2_AA/Run/Checkpoints/8751/Cp-VarsGr_SphereA0.h5
/panfs/ds09/sxs/himanshu/runs/simple_BBH_for_GH_testing/one_high_spin_zero_ecc_equal_mass/Ev/Lev2_AA/Run/Checkpoints/11653/Cp-VarsGr_SphereA0.h5
/panfs/ds09/sxs/himanshu/runs/simple_BBH_for_GH_testing/one_high_spin_zero_ecc_equal_mass/Ev/Lev3_AA/Run/Checkpoints/6799/Cp-VarsGr_SphereA0.h5
/panfs/ds09/sxs/himanshu/runs/simple_BBH_for_GH_testing/one_high_spin_zero_ecc_equal_mass/Ev/Lev3_AA/Run/Checkpoints/13778/Cp-VarsGr_SphereA0.h5
/panfs/ds09/sxs/himanshu/runs/simple_BBH_for_GH_testing/one_high_spin_zero_ecc_equal_mass/Ev/Lev1_AB/Run/Checkpoints/18758/Cp-VarsGr_

In [8]:
with open("./find_paths.ouput",'w') as f:
  for path in path_collection:
    f.write(path+" ")

In [60]:

path_pattern = checkpoints_A0_glob
path_collection = []

def find_time(file_path):
    """" Reads time from Cp-EvolutionLoopControl.txt"""
    with open(file_path,'r') as f:
        a = f.readline()
        return float(a.split(";")[0].split("=")[1])
     


for folder_name in glob.iglob(path_pattern, recursive=True):
    if os.path.isdir(folder_name) or os.path.isfile(folder_name):
        time = find_time(folder_name[:-21]+"Cp-EvolutionLoopControl.txt")
        time_and_path = [time,folder_name]
        path_collection.append(time_and_path)
        # print(time_and_path)


time_sorted_path_list = list(pd.DataFrame(path_collection,columns=["Time","Path"]).sort_values(by=['Time'])['Path'])
time_sorted_path_list


with open("./find_paths.output",'w') as f:
  for path in time_sorted_path_list:
    f.write(path+" ")

In [30]:
file_name = "/panfs/ds09/sxs/himanshu/runs/simple_BBH_for_GH_testing/one_high_spin_zero_ecc_equal_mass/Ev/Lev3_AA/Run/Checkpoints/13778/Cp-EvolutionLoopControl.txt"

In [49]:
"/panfs/ds09/sxs/himanshu/runs/simple_BBH_for_GH_testing/one_high_spin_zero_ecc_equal_mass/Ev/Lev3_AF/Run/Checkpoints/27660/Cp-VarsGr_SphereA0.h5"[:-21]

'/panfs/ds09/sxs/himanshu/runs/simple_BBH_for_GH_testing/one_high_spin_zero_ecc_equal_mass/Ev/Lev3_AF/Run/Checkpoints/27660/'

In [46]:
len("Cp-VarsGr_SphereA0.h5")

21

In [38]:
float(time)

344.48128012719457

In [39]:
time

'   3.4448128012719457e+02'